<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/personalidad/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 01: Carga de datos

## Importando base de datos

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = 'https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing'
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet('Respuestas de formulario 1').get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

## Inspección del data frame

In [ ]:
type(df)

In [ ]:
# ver el tamaño del df
df.shape

In [ ]:
# Ver las primeras filas
df.head()

In [ ]:
# Ver al estructura del df
df.info()

In [ ]:
# dtype
df['Sexo'].dtype

In [ ]:
# Número de filas
len(df)

In [ ]:
# Número de columnas
len(df.columns)

# Tema 02: Transformación de datos

## Valores perdidos

### Identificando NAs

In [ ]:
# Los NAs están como strings vacíos
import numpy as np
df.replace('', np.nan, inplace=True)

In [ ]:
df['Escribe tu edad exacta'] \
.isna() \
.value_counts()

### Tratando los NAs

In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors='coerce')

In [ ]:
df['Escribe tu edad exacta'].dtype

Reemplazo por la media

In [ ]:
edad_promedio = df['Escribe tu edad exacta'].mean()

In [ ]:
# Creando df2
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2['edad2'] = df2['Escribe tu edad exacta'].fillna(edad_promedio)

In [ ]:
df2[['Escribe tu edad exacta', 'edad2']]

Reubicando edad2

In [ ]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [ ]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove('edad2')

In [ ]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index('Escribe tu edad exacta')

In [ ]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index('Escribe tu edad exacta') + 1, 'edad2')

In [ ]:
df2 = df2[lista_columnas]

In [ ]:
df2.info()

Creando relocate() en py

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = 'edad2',
    after = 'Escribe tu edad exacta'
)

Eliminar la fila completa

In [ ]:
df2.shape

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.shape

## Estandarización de variables

### Normalización

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instansciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[['edad2']])

In [ ]:
# creando df3
df3 = df2.copy()

In [ ]:
df3['edadZ'] = normalizador.fit_transform(df3[['edad2']])

In [ ]:
df3 = relocate(
  df = df3,
  columna = 'edadZ',
  after = 'edad2'
)

### Rango

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
df3['edadR'] = rango.fit_transform(df3[['edad2']])

In [ ]:
df3 = relocate(
  df = df3,
  columna = 'edadR',
  after = 'edadZ'
)

In [ ]:
df3[['edad2', 'edadZ', 'edadR']]

## Agrupaciones

### Numéricas

In [ ]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ['18 o menos', '19 a 21', 'Más de 21']

In [ ]:
df3['edadGR'] = pd.cut(
    x = df3['edad2'],
    bins = cortes,
    labels = etiquetas
)

In [ ]:
df3 = relocate(
    df = df3,
    columna = 'edadGR',
    after = 'edadR'
)

In [ ]:
df3['edadGR'].value_counts()

### Categóricas

In [ ]:
# Revisando índices
df3.info()

In [ ]:
# Valores únicos
pd.unique(df3.iloc[:,8])

In [ ]:
# Frecuencias
df3.iloc[:,8].value_counts()

In [ ]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero'])

Dummitización

In [ ]:
# Función condicional normal
def top2box(x):
  # if x == 'Totalmente verdadero' or x == 'Un poco verdadero':
  if x in ['Totalmente verdadero', 'Un poco verdadero']:
    return 1
  else:
    return 0

In [ ]:
# Aplicando la función
df3.iloc[:,8] \
.apply(top2box) \
.value_counts()

In [ ]:
# Función lambda
df3.iloc[:,8].apply(
    lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0
)

In [ ]:
# Frecuencias
df3.iloc[:,8] \
.apply(lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0) \
.value_counts()

### Ordinales

In [ ]:
import numpy as np

In [ ]:
pd.unique(df3.iloc[:,8])

In [ ]:
# Cuando las codiciones son muchas (5 en esta caso), se recomienda usar select()
condiciones = [
    df3.iloc[:,8] == 'Totalmente verdadero',
    df3.iloc[:,8] == 'Un poco verdadero',
    df3.iloc[:,8] == 'No lo sé',
    df3.iloc[:,8] == 'Un poco falso',
    df3.iloc[:,8] == 'Totalmente falso',
]

# Hacer coincidir el orden
opciones = [5,4,3,2,1]

In [ ]:
# Usando select()
ordinal = np.select(
    condlist = condiciones,
    choicelist = opciones,
    default = np.nan
)

In [ ]:
ordinal

In [ ]:
ordinal.mean()

In [ ]:
np.unique(ordinal, return_counts = True)

### Bucles

Forma standard

In [ ]:
# creando df4
df4 = df3.copy()

In [ ]:
df4.columns

In [ ]:
# crear una lista vacía
frases = []

# se ejecuta el bucle para llenar la lista
for col in df4.columns:
  if col.startswith('Según tu'):
    frases.append(col)

In [ ]:
frases

In [ ]:
# aplicando el bucle para dummitizar, recordemos que creamos la función top2box()
for frase in frases:
  df4[frase] = df4[frase].apply(top2box)

In [ ]:
df4[frases]

Forma pythonica

In [ ]:
# Creando df5
df5 = df3.copy()

In [ ]:
# lista vacía
frases = []

# comprehension list
frases = [col for col in df5.columns if col.startswith('Según tu')]

In [ ]:
# aplicando el bucle, esta vez con una lambda
for frase in frases:
  df5[frase] = df5[frase].apply(lambda x: 1 if x in ['Totalmente verdadero', 'Un poco verdadero'] else 0)

In [ ]:
df5[frases] == df4[frases]

# Tema 03: Manipulación de datos

## Selección de columnas

In [ ]:
df5['Sexo']

In [ ]:
# extraer como df de una sola columna
df5[['Sexo']]

In [ ]:
df5[['Sexo', 'Escribe tu edad exacta']]

In [ ]:
# eliminar columnas
df5.drop(columns = 'Marca temporal')

In [ ]:
# seleccionar cols que comienzan con...
df5[[col for col in df5.columns if col.startswith('¿Cuánto')]]

In [ ]:
# seleccionar cols que terminan con...
df5[[col for col in df5.columns if col.endswith(']')]]

In [ ]:
# contains
df5.filter(like = 'edad', axis = 1)

## Selección de filas

In [ ]:
# Seleccionar las filas cuando Sexo = 'Mujer'
df5[df5['Sexo'] == 'Mujer'].head()

In [ ]:
# Seleccionar las filas cuando Sexo no es igual a 'Hombre'
df5[df5['Sexo'] != 'Hombre'].head()

In [ ]:
# Seleccionar filas cuando edad2 es mayor o igual a 20
df5[df5['edad2'] >= 20].head()

In [ ]:
# Seleccionar filas cuando edad2 está entre 18 y 21
df5[(df5['edad2'] >= 18) & (df5['edad2'] <= 21)].head()

In [ ]:
# Seleccionar filas cuando edad está entre 18 y 21 y cuando sexo = 'Mujer'
df5[
    (df5['edad2'] >= 18) &
    (df5['edad2'] <= 21) &
    (df5['Sexo'] == 'Mujer')
].head()

In [ ]:
df2[
    (df2['edad2'] > 20) &
    (df2['Sexo'] == 'Hombre') &
    (df2.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero']))
].head()

## Renombrando columnas

In [115]:
df6 = df5.copy()

In [ ]:
df6.columns

APPS

**Paso 1:** Crear una lista con los nuevos nombres

In [119]:
apps = ['TikTok', 'Instagram', 'Facebook', 'YouTube']

**Paso 2:** Crear dict que contenga los nombres antiguos como *keys* y nuevos como *values*

In [124]:
apps_dict = dict(zip(
    [col for col in df6.columns if col.startswith('¿Cuánto')],
    apps
))

In [126]:
apps_dict

{'¿Cuánto tiempo has pasado en los últimos 7 días en TikTok? Si no lo las usado coloca 00:00': 'TikTok',
 '¿Cuánto tiempo has pasado en los últimos 7 días en Instagram? Si no lo las usado coloca 00:00': 'Instagram',
 '¿Cuánto tiempo has pasado en los últimos 7 días en Facebook? Si no lo las usado coloca 00:00': 'Facebook',
 '¿Cuánto tiempo has pasado en los últimos 7 días en YouTube? Si no lo las usado coloca 00:00': 'YouTube'}

**Paso 3:** Renombrando

In [127]:
df6.rename(apps_dict, axis = 1, inplace = True)

In [ ]:
df6.info()

Frases

**Paso 0:** Convertir la lista en Pandas Series

In [ ]:
type(frases)

In [130]:
frases2 = pd.Series(frases)

In [ ]:
type(frases2)

**Paso 1:** Crear una lista con los nuevos nombres

In [138]:
frases2 = frases2.str.extract('\[(.+)\]')[0]

In [ ]:
frases2

**Paso 2:** Crear dict que contenga los nombres antiguos como *keys* y nuevos como *values*

In [140]:
frases_dict = dict(zip(frases, frases2))

**Pase 3:** Renombrando

In [141]:
df6.rename(frases_dict, axis=1, inplace=True)

In [ ]:
df6.info()

## Pivotado

### Pivot longer

In [143]:
apps

['TikTok', 'Instagram', 'Facebook', 'YouTube']

In [146]:
df7 = df6.melt(
    id_vars = ['Marca temporal', 'Sexo', 'edad2'],
    value_vars = apps,
    var_name = 'app',
    value_name = 'time'
)

In [147]:
df7.head()

,Marca temporal,Sexo,edad2,app,time
0,9/08/2021 12:23:46,Hombre,16.0,TikTok,0:00:00
1,9/08/2021 12:25:01,Mujer,16.0,TikTok,0:50:00
2,9/08/2021 12:26:31,Hombre,17.0,TikTok,8:00:00
3,9/08/2021 12:28:49,Mujer,22.0,TikTok,22:35:00
4,9/08/2021 12:31:43,Hombre,16.0,TikTok,7:00:00


### Pivot wider

In [153]:
df8 = df7.pivot(
    index = ['Marca temporal', 'Sexo', 'edad2'],
    columns = 'app',
    values = 'time'
).reset_index()

In [154]:
df8.head()

app,Marca temporal,Sexo,edad2,Facebook,Instagram,TikTok,YouTube
0,10/01/2023 18:16:15,Mujer,25.0,0:00:00,6:00:00,0:20:00,3:00:00
1,10/01/2023 18:16:58,Mujer,21.0,5:00:00,6:00:00,3:00:00,2:00:00
2,10/01/2023 18:17:15,Hombre,22.0,5:00:00,1:00:00,3:00:00,3:00:00
3,10/01/2023 18:17:22,Mujer,22.0,1:50:00,6:33:00,7:52:00,0:54:00
4,10/01/2023 18:17:33,Mujer,25.0,0:37:00,2:15:00,1:28:00,0:00:00


# Tema 04: Outliers